<a href="https://colab.research.google.com/github/harsh-agar/E-Net/blob/master/Frustum_V2_func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu==1.13.1
!pip install hyperopt

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# ##Test TF-GPU is working fine
# # Creates a graph.
# with tf.device('/gpu:0'):
#   a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
#   b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
# c = tf.matmul(a, b)
# # Creates a session with log_device_placement set to True.
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# # Runs the op.
# print(sess.run(c))

In [0]:
import os
os.chdir('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/pointnet2/tf_ops/sampling')
!bash tf_sampling_compile.sh
os.chdir('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master//pointnet2/tf_ops/grouping')
!bash tf_grouping_compile.sh
os.chdir('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master//pointnet2/tf_ops/3d_interpolation')
!bash tf_interpolate_compile.sh

In [0]:
import os
import sys
BASE_DIR = os.path.dirname('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/pointnet2/')
ROOT_DIR = os.path.dirname('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/pointnet2/utils/')
sys.path.append(BASE_DIR)
sys.path.append(ROOT_DIR)
sys.path.append('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/pointnet2/tf_ops/sampling/')
sys.path.append('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/pointnet2/tf_ops/grouping/')
sys.path.append('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/pointnet2/tf_ops/3d_interpolation/')

In [0]:
from __future__ import print_function

import tensorflow as tf
import tf_util
import numpy as np
import importlib
import csv
from pointnet_util import pointnet_sa_module, pointnet_sa_module_msg, pointnet_fp_module

In [0]:
NUM_HEADING_BIN = 12
NUM_SIZE_CLUSTER = 8 # one cluster for each type
VAl_CONSTANT = 3 # 3 if 'x,y,z', 2 if 'x,y'
VAL_NUM_HEADING_BIN = 2
VAL_NUM_SIZE_CLUSTER = 4 # 4 if 'x,y,z', 3 if 'x,y'

# Set training configurations
BATCH_SIZE = 1
NUM_POINT = 1024
NUM_CLASSES = 2
NUM_CHANNEL = 4
NUM_OBJECT_POINT = 512
xyz_only = 1

batch_size = 1

In [0]:
def tf_gather_object_pc(point_cloud, mask, npoints=512):
	''' Gather object point clouds according to predicted masks.
	Input:
		point_cloud: TF tensor in shape (B,N,C)
		mask: TF tensor in shape (B,N) of 0 (not pick) or 1 (pick)
		npoints: int scalar, maximum number of points to keep (default: 512)
	Output:
		object_pc: TF tensor in shape (B,npoint,C)
		indices: TF int tensor in shape (B,npoint,2)
	'''
	def mask_to_indices(mask):
		indices = np.zeros((mask.shape[0], npoints, 2), dtype=np.int32)
		for i in range(mask.shape[0]):
			pos_indices = np.where(mask[i,:]>0.5)[0] #BxN
			# skip cases when pos_indices is empty
			if len(pos_indices) > 0: 
				if len(pos_indices) > npoints:
					choice = np.random.choice(len(pos_indices),
						npoints, replace=False)
				else:
					choice = np.random.choice(len(pos_indices),
						npoints-len(pos_indices), replace=True)
					choice = np.concatenate((np.arange(len(pos_indices)), choice))
				np.random.shuffle(choice)
				indices[i,:,1] = pos_indices[choice]
			indices[i,:,0] = i
		return indices

	indices = tf.py_func(mask_to_indices, [mask], tf.int32)  
	object_pc = tf.gather_nd(point_cloud, indices)
	return object_pc, indices

In [0]:
def get_instance_seg_v2_net(point_cloud, is_training, bn_decay, end_points):
  l0_xyz = tf.slice(point_cloud, [0,0,0], [-1,-1,3])
  l0_points = tf.slice(point_cloud, [0,0,3], [-1,-1,1])

  # Set abstraction layers
  l1_xyz, l1_points = pointnet_sa_module_msg(l0_xyz, l0_points,
      128, [0.2,0.4,0.8], [32,64,128],
      [[32,32,64], [64,64,128], [64,96,128]],
      is_training, bn_decay, scope='layer1')
  l2_xyz, l2_points = pointnet_sa_module_msg(l1_xyz, l1_points,
      32, [0.4,0.8,1.6], [64,64,128],
      [[64,64,128], [128,128,256], [128,128,256]],
      is_training, bn_decay, scope='layer2')
  l3_xyz, l3_points, _ = pointnet_sa_module(l2_xyz, l2_points,
      npoint=None, radius=None, nsample=None, mlp=[128,256,1024],
      mlp2=None, group_all=True, is_training=is_training,
      bn_decay=bn_decay, scope='layer3')
  net = tf_util.fully_connected(tf.reshape(l3_points, [batch_size, 1024]), 512, scope='fc1-classification', bn=True, is_training=is_training, bn_decay=bn_decay)
  net = tf_util.fully_connected(net, 256, scope='fc2-classification', bn=True,
      is_training=is_training, bn_decay=bn_decay)
  predicted_classes_scores = tf_util.fully_connected(net, NUM_CLASSES, activation_fn=tf.nn.softmax,
      scope='fc3-classification')

  l3_points = tf.concat([l3_points, tf.expand_dims(predicted_classes_scores, 1)], axis=2)

  # Segmentation
  l2_points = pointnet_fp_module(l2_xyz, l3_xyz, l2_points, l3_points,
      [128,128], is_training, bn_decay, scope='fa_layer1')
  l1_points = pointnet_fp_module(l1_xyz, l2_xyz, l1_points, l2_points,
      [128,128], is_training, bn_decay, scope='fa_layer2')
  l0_points = pointnet_fp_module(l0_xyz, l1_xyz,
      tf.concat([l0_xyz,l0_points],axis=-1), l1_points,
      [128,128], is_training, bn_decay, scope='fa_layer3')

  # FC layers
  net = tf_util.conv1d(l0_points, 128, 1, padding='VALID', bn=True,
      is_training=is_training, scope='conv1d-fc1', bn_decay=bn_decay)
  end_points['feats'] = net 
  net = tf_util.dropout(net, keep_prob=0.7,
      is_training=is_training, scope='dp1')
  logits = tf_util.conv1d(net, 2, 1,
      padding='VALID', activation_fn=None, scope='conv1d-fc2')
  return predicted_classes_scores, logits, end_points

In [0]:
def point_cloud_masking(point_cloud, logits, end_points, xyz_only=True):
    num_point = point_cloud.get_shape()[1].value
    batch_size = point_cloud.get_shape()[0].value
    mask = tf.slice(logits,[0,0,0],[-1,-1,1]) < \
        tf.slice(logits,[0,0,1],[-1,-1,1]) #True when object is present else false
    mask = tf.cast(mask, tf.float32) # BxNx1
    mask_count = tf.tile(tf.reduce_sum(mask,axis=1,keep_dims=True),
        [1,1,3]) # Bx1x3
    point_cloud_xyz = tf.slice(point_cloud, [0,0,0], [-1,-1,3]) # BxNx3
    mask_xyz_mean = tf.reduce_sum(tf.tile(mask, [1,1,3])*point_cloud_xyz,
        axis=1, keep_dims=True) # Bx1x3 ##Computes mean of all the mask points 'x,y,z' which have a object 'mask' present in it
    mask = tf.squeeze(mask, axis=[2]) # BxN
    mask_xyz_mean = mask_xyz_mean/tf.maximum(mask_count,1) # Bx1x3

    # Translate to masked points' centroid
    point_cloud_xyz_stage1 = point_cloud_xyz - \
        tf.tile(mask_xyz_mean, [1,num_point,1])
    xyz_only = 1
    if xyz_only: 
        point_cloud_stage1 = point_cloud_xyz_stage1
    else:
        point_cloud_features = tf.slice(point_cloud, [0,0,3], [-1,-1,-1])
        point_cloud_stage1 = tf.concat(\
            [point_cloud_xyz_stage1, point_cloud_features], axis=-1)
    num_channels = point_cloud_stage1.get_shape()[2].value
    object_point_cloud, _ = tf_gather_object_pc(point_cloud_stage1,
        mask, NUM_OBJECT_POINT)
    object_point_cloud.set_shape([batch_size, NUM_OBJECT_POINT, num_channels])
    return object_point_cloud, tf.squeeze(mask_xyz_mean, axis=1), end_points

In [0]:
def get_center_regression_net(object_point_cloud, predicted_classes_scores,
                              is_training, bn_decay, end_points):
    num_point = object_point_cloud.get_shape()[1].value
    net = tf.expand_dims(object_point_cloud, 2)
    net = tf_util.conv2d(net, 128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv-reg1-stage1', bn_decay=bn_decay)
    net = tf_util.conv2d(net,  128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv-reg2-stage1', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 256, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv-reg3-stage1', bn_decay=bn_decay)
    net = tf_util.max_pool2d(net, [num_point,1],
        padding='VALID', scope='maxpool-stage1')
    net = tf.squeeze(net, axis=[1,2])
    net = tf.concat([net, predicted_classes_scores], axis=1)
    net = tf_util.fully_connected(net, 256, scope='fc1-stage1', bn=True,
        is_training=is_training, bn_decay=bn_decay)
    net = tf_util.fully_connected(net, 128, scope='fc2-stage1', bn=True,
        is_training=is_training, bn_decay=bn_decay)
    predicted_center = tf_util.fully_connected(net, 3, activation_fn=None,
        scope='fc3-stage1')
    return predicted_center, end_points

In [0]:
def get_3d_box_estimation_v2_net(object_point_cloud, predicted_classes_scores, is_training, bn_decay, end_points):
  batch_size = object_point_cloud.get_shape()[0].value

  l0_xyz = object_point_cloud
  l0_points = None
  # Set abstraction layers
  l1_xyz, l1_points, l1_indices = pointnet_sa_module(l0_xyz, l0_points,
      npoint=128, radius=0.2, nsample=64, mlp=[64,64,128],
      mlp2=None, group_all=False,
      is_training=is_training, bn_decay=bn_decay, scope='ssg-layer1')
  l2_xyz, l2_points, l2_indices = pointnet_sa_module(l1_xyz, l1_points,
      npoint=32, radius=0.4, nsample=64, mlp=[128,128,256],
      mlp2=None, group_all=False,
      is_training=is_training, bn_decay=bn_decay, scope='ssg-layer2')
  l3_xyz, l3_points, l3_indices = pointnet_sa_module(l2_xyz, l2_points,
      npoint=None, radius=None, nsample=None, mlp=[256,256,512],
      mlp2=None, group_all=True,
      is_training=is_training, bn_decay=bn_decay, scope='ssg-layer3')

  # Fully connected layers
  net = tf.reshape(l3_points, [batch_size, -1])
  net = tf.concat([net, predicted_classes_scores], axis=1)
  net = tf_util.fully_connected(net, 512, bn=True,
      is_training=is_training, scope='fc1', bn_decay=bn_decay)
  net = tf_util.fully_connected(net, 256, bn=True,
      is_training=is_training, scope='fc2', bn_decay=bn_decay)

  # The first 3 numbers: box center coordinates (cx,cy,cz),
  # the next NUM_HEADING_BIN*2:  heading bin class scores and bin residuals
  # next NUM_SIZE_CLUSTER*4: box cluster scores and residuals
  output = tf_util.fully_connected(net,
      VAl_CONSTANT+VAL_NUM_HEADING_BIN*NUM_HEADING_BIN+VAL_NUM_SIZE_CLUSTER*NUM_SIZE_CLUSTER, activation_fn=None, scope='fc3')
  return output, end_points
#   return l3_points, end_points

In [0]:
def parse_output_to_tensors(output, end_points):
    ''' Parse batch output to separate tensors (added to end_points)
    Input:
        output: TF tensor in shape (B,VAl_CONSTANT+VAL_NUM_HEADING_BIN*NUM_HEADING_BIN+VAL_NUM_SIZE_CLUSTER*NUM_SIZE_CLUSTER)
        end_points: dict
    Output:
        end_points: dict (updated)
    '''
    batch_size = output.get_shape()[0].value
    center = tf.slice(output, [0,0], [-1,VAl_CONSTANT])
    end_points['center_boxnet'] = center

    heading_scores = tf.slice(output, [0,VAl_CONSTANT], [-1,NUM_HEADING_BIN])
    heading_residuals_normalized = tf.slice(output, [0,VAl_CONSTANT+NUM_HEADING_BIN],
        [-1,NUM_HEADING_BIN])
    end_points['heading_scores'] = heading_scores # BxNUM_HEADING_BIN
    end_points['heading_residuals_normalized'] = \
        heading_residuals_normalized # BxNUM_HEADING_BIN (-1 to 1)
    end_points['heading_residuals'] = \
        heading_residuals_normalized * (np.pi/NUM_HEADING_BIN) # BxNUM_HEADING_BIN
    
    size_scores = tf.slice(output, [0,VAl_CONSTANT+NUM_HEADING_BIN*VAL_NUM_HEADING_BIN],
        [-1,NUM_SIZE_CLUSTER]) # BxNUM_SIZE_CLUSTER
    size_residuals_normalized = tf.slice(output,
        [0,VAl_CONSTANT+NUM_HEADING_BIN*VAL_NUM_HEADING_BIN+NUM_SIZE_CLUSTER], [-1,NUM_SIZE_CLUSTER*(VAL_NUM_SIZE_CLUSTER-1)])
    size_residuals_normalized = tf.reshape(size_residuals_normalized,
        [batch_size, NUM_SIZE_CLUSTER, (VAL_NUM_SIZE_CLUSTER-1)]) # BxNUM_SIZE_CLUSTERx3
    end_points['size_scores'] = size_scores
    end_points['size_residuals_normalized'] = size_residuals_normalized
    '''
    end_points['size_residuals'] = size_residuals_normalized * \
        tf.expand_dims(tf.constant(g_mean_size_arr, dtype=tf.float32), 0)
    '''
    end_points['size_residuals'] = size_residuals_normalized
    return end_points

In [0]:
def get_model(point_cloud, is_training, bn_decay=None):
    end_points = {}
    # 3D Instance Segmentation PointNet
    predicted_classes_scores, logits, end_points = get_instance_seg_v2_net(point_cloud, is_training, bn_decay, end_points)
    end_points['mask_logits'] = logits
    end_points['class_logits'] = predicted_classes_scores
    # Masking
    # select masked points and translate to masked points' centroid
    object_point_cloud_xyz, mask_xyz_mean, end_points = \
        point_cloud_masking(point_cloud, logits, end_points)

    # T-Net and coordinate translation
    center_delta, end_points = get_center_regression_net(\
        object_point_cloud_xyz, predicted_classes_scores,
        is_training, bn_decay, end_points)
    stage1_center = center_delta + mask_xyz_mean # Bx3
    end_points['stage1_center'] = stage1_center
    # Get object point cloud in object coordinate
    object_point_cloud_xyz_new = \
        object_point_cloud_xyz - tf.expand_dims(center_delta, 1)

#     Amodel Box Estimation PointNet
    output, end_points = get_3d_box_estimation_v2_net(\
        object_point_cloud_xyz_new, predicted_classes_scores,
        is_training, bn_decay, end_points)

    # Parse output to 3D box parameters
    end_points = parse_output_to_tensors(output, end_points)
    end_points['center'] = end_points['center_boxnet'] + stage1_center # Bx3

    return end_points


In [97]:
BATCH_SIZE = 1
NUM_POINT = 1024
# MODEL = importlib.import_module('frustum_pointnets_v1')
NUM_CLASSES = 2
NUM_CHANNEL = 4
batch_size = 1

tf.reset_default_graph()
with tf.Graph().as_default():
  is_training_pl = tf.placeholder(tf.bool, shape=())
  pointclouds_pl = tf.placeholder(tf.float32, shape=(BATCH_SIZE, NUM_POINT, 6))
  batch_size = pointclouds_pl.get_shape()[0].value
  num_point = pointclouds_pl.get_shape()[1].value        
  is_training = tf.constant(False, dtype=tf.bool)

  end_points = get_model(pointclouds_pl, is_training_pl)

  sess = tf.Session()
  # with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  pc = np.loadtxt(open("/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/data_sample.csv", "rb"), delimiter=",")
  ops = {'pointclouds_pl': pointclouds_pl,
           'is_training_pl': is_training_pl}
  feed_dict = {ops['pointclouds_pl']: [pc], ops['is_training_pl']: False}

  end_points = sess.run(end_points, feed_dict=feed_dict)
  print(end_points)

  for key, value in end_points.items() :
      print (key)
      print (value.shape)
      print ("\n")

{'feats': array([[[0.        , 3.2328126 , 3.2665458 , ..., 0.        ,
         0.        , 0.        ],
        [0.23080653, 1.5711966 , 2.96782   , ..., 0.90524364,
         0.6365163 , 0.        ],
        [0.        , 4.953106  , 2.333268  , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.6596206 , 1.72573   , 1.6862521 , ..., 0.6372791 ,
         0.2695727 , 0.        ],
        [0.        , 2.4600356 , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 2.7059364 , 0.        , ..., 0.        ,
         0.        , 0.        ]]], dtype=float32), 'mask_logits': array([[[ 0.7834685 , -2.5934873 ],
        [ 1.0725826 , -1.4713265 ],
        [ 1.1804701 , -4.087547  ],
        ...,
        [ 0.62612945, -1.2759098 ],
        [ 0.55096453, -5.277473  ],
        [ 0.88969475, -3.034563  ]]], dtype=float32), 'class_logits': array([[0.8410748 , 0.15892518]], dtype=float32), 'stage1_center': array([[-1.0210165, -1.8655769, -0.540

In [0]:
# tf.reset_default_graph()
# with tf.Graph().as_default():
# #   with tf.device('/gpu:0'):
#   is_training = tf.constant(False, dtype=tf.bool)
#   bn_decay=None
#   point_cloud = tf.placeholder(tf.float32, shape=(BATCH_SIZE, NUM_POINT, 6))
#   batch_size = point_cloud.get_shape()[0].value
#   num_point = point_cloud.get_shape()[1].value
#   # Classification
#   l0_xyz = tf.slice(point_cloud, [0,0,0], [-1,-1,3])
#   l0_points = tf.slice(point_cloud, [0,0,3], [-1,-1,1])

#   # Set abstraction layers
#   l1_xyz, l1_points = pointnet_sa_module_msg(l0_xyz, l0_points,
#       128, [0.2,0.4,0.8], [32,64,128],
#       [[32,32,64], [64,64,128], [64,96,128]],
#       is_training, bn_decay, scope='layer1')
#   l2_xyz, l2_points = pointnet_sa_module_msg(l1_xyz, l1_points,
#       32, [0.4,0.8,1.6], [64,64,128],
#       [[64,64,128], [128,128,256], [128,128,256]],
#       is_training, bn_decay, scope='layer2')
#   l3_xyz, l3_points, _ = pointnet_sa_module(l2_xyz, l2_points,
#       npoint=None, radius=None, nsample=None, mlp=[128,256,1024],
#       mlp2=None, group_all=True, is_training=is_training,
#       bn_decay=bn_decay, scope='layer3')
#   net = tf_util.fully_connected(tf.reshape(l3_points, [batch_size, 1024]), 512, scope='fc1-classification', bn=True, is_training=is_training, bn_decay=bn_decay)
#   net = tf_util.fully_connected(net, 256, scope='fc2-classification', bn=True,
#       is_training=is_training, bn_decay=bn_decay)
#   predicted_classes_scores = tf_util.fully_connected(net, NUM_CLASSES, activation_fn=tf.nn.softmax,
#       scope='fc3-classification')

#   l3_points = tf.concat([l3_points, tf.expand_dims(predicted_classes_scores, 1)], axis=2)

#   # Segmentation
#   l2_points = pointnet_fp_module(l2_xyz, l3_xyz, l2_points, l3_points,
#       [128,128], is_training, bn_decay, scope='fa_layer1')
#   l1_points = pointnet_fp_module(l1_xyz, l2_xyz, l1_points, l2_points,
#       [128,128], is_training, bn_decay, scope='fa_layer2')
#   l0_points = pointnet_fp_module(l0_xyz, l1_xyz,
#       tf.concat([l0_xyz,l0_points],axis=-1), l1_points,
#       [128,128], is_training, bn_decay, scope='fa_layer3')

#   # FC layers
#   net = tf_util.conv1d(l0_points, 128, 1, padding='VALID', bn=True,
#       is_training=is_training, scope='conv1d-fc1', bn_decay=bn_decay)
#   net = tf_util.dropout(net, keep_prob=0.7,
#       is_training=is_training, scope='dp1')
#   logits = tf_util.conv1d(net, 2, 1,
#       padding='VALID', activation_fn=None, scope='conv1d-fc2')

#   # Masking
#   mask = tf.slice(logits,[0,0,0],[-1,-1,1]) < \
#     tf.slice(logits,[0,0,1],[-1,-1,1]) #True when object is present else false
#   mask = tf.cast(mask, tf.float32) # BxNx1
#   mask_count = tf.tile(tf.reduce_sum(mask,axis=1,keep_dims=True),
#     [1,1,3]) # Bx1x3
#   point_cloud_xyz = tf.slice(point_cloud, [0,0,0], [-1,-1,3]) # BxNx3
#   mask_xyz_mean = tf.reduce_sum(tf.tile(mask, [1,1,3])*point_cloud_xyz,
#     axis=1, keep_dims=True) # Bx1x3 ##Computes mean of all the mask points 'x,y,z' which have a object 'mask' present in it
#   mask = tf.squeeze(mask, axis=[2]) # BxN
#   mask_xyz_mean = mask_xyz_mean/tf.maximum(mask_count,1) # Bx1x3

#   # Translate to masked points' centroid
#   point_cloud_xyz_stage1 = point_cloud_xyz - \
#     tf.tile(mask_xyz_mean, [1,num_point,1])
#   if xyz_only: 
#     point_cloud_stage1 = point_cloud_xyz_stage1
#   else:
#     point_cloud_features = tf.slice(point_cloud, [0,0,3], [-1,-1,-1])
#     point_cloud_stage1 = tf.concat(\
#       [point_cloud_xyz_stage1, point_cloud_features], axis=-1)
#   num_channels = point_cloud_stage1.get_shape()[2].value
#   object_point_cloud, _ = tf_gather_object_pc(point_cloud_stage1,
#     mask, NUM_OBJECT_POINT)
#   object_point_cloud.set_shape([batch_size, NUM_OBJECT_POINT, num_channels])
#   #T-Net
#   num_point = object_point_cloud.get_shape()[1].value
#   net = tf.expand_dims(object_point_cloud, 2)
#   net = tf_util.conv2d(net, 128, [1,1],
#              padding='VALID', stride=[1,1],
#              bn=True, is_training=is_training,
#              scope='conv-reg1-stage1', bn_decay=bn_decay)
#   net = tf_util.conv2d(net,  128, [1,1],
#              padding='VALID', stride=[1,1],
#              bn=True, is_training=is_training,
#              scope='conv-reg2-stage1', bn_decay=bn_decay)
#   net = tf_util.conv2d(net, 256, [1,1],
#              padding='VALID', stride=[1,1],
#              bn=True, is_training=is_training,
#              scope='conv-reg3-stage1', bn_decay=bn_decay)
#   net = tf_util.max_pool2d(net, [num_point,1],
#     padding='VALID', scope='maxpool-stage1')
#   net = tf.squeeze(net, axis=[1,2])
#   net = tf.concat([net, predicted_classes_scores], axis=1)
#   net = tf_util.fully_connected(net, 256, scope='fc1-stage1', bn=True,
#     is_training=is_training, bn_decay=bn_decay)
#   net = tf_util.fully_connected(net, 128, scope='fc2-stage1', bn=True,
#     is_training=is_training, bn_decay=bn_decay)
#   predicted_center = tf_util.fully_connected(net, 3, activation_fn=None,
#     scope='fc3-stage1')
#   # Translation
#   stage1_center = predicted_center + mask_xyz_mean # Bx3

#   # Get object point cloud in object coordinate
#   object_point_cloud_xyz_new = \
#     object_point_cloud - tf.expand_dims(predicted_center, 1)  
  
#   # Amodal
#   batch_size = object_point_cloud_xyz_new.get_shape()[0].value

#   l0_xyz = object_point_cloud_xyz_new
#   l0_points = None
#   # Set abstraction layers
#   l1_xyz, l1_points, l1_indices = pointnet_sa_module(l0_xyz, l0_points,
#       npoint=128, radius=0.2, nsample=64, mlp=[64,64,128],
#       mlp2=None, group_all=False,
#       is_training=is_training, bn_decay=bn_decay, scope='ssg-layer1')
#   l2_xyz, l2_points, l2_indices = pointnet_sa_module(l1_xyz, l1_points,
#       npoint=32, radius=0.4, nsample=64, mlp=[128,128,256],
#       mlp2=None, group_all=False,
#       is_training=is_training, bn_decay=bn_decay, scope='ssg-layer2')
#   l3_xyz, l3_points, l3_indices = pointnet_sa_module(l2_xyz, l2_points,
#       npoint=None, radius=None, nsample=None, mlp=[256,256,512],
#       mlp2=None, group_all=True,
#       is_training=is_training, bn_decay=bn_decay, scope='ssg-layer3')

#   # Fully connected layers
#   net = tf.reshape(l3_points, [batch_size, -1])
#   net = tf.concat([net, predicted_classes_scores], axis=1)
#   net = tf_util.fully_connected(net, 512, bn=True,
#       is_training=is_training, scope='fc1', bn_decay=bn_decay)
#   net = tf_util.fully_connected(net, 256, bn=True,
#       is_training=is_training, scope='fc2', bn_decay=bn_decay)

#   # The first 3 numbers: box center coordinates (cx,cy,cz),
#   # the next NUM_HEADING_BIN*2:  heading bin class scores and bin residuals
#   # next NUM_SIZE_CLUSTER*4: box cluster scores and residuals
#   output = tf_util.fully_connected(net,
#       VAl_CONSTANT+VAL_NUM_HEADING_BIN*NUM_HEADING_BIN+VAL_NUM_SIZE_CLUSTER*NUM_SIZE_CLUSTER, activation_fn=None, scope='fc3')  
#   #################################

#   sess = tf.Session()
#   # with tf.Session() as sess:
#   sess.run(tf.global_variables_initializer())
#   pc = np.loadtxt(open("/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/data_sample.csv", "rb"), delimiter=",")
# #   print (pc)
#   ops = {'pointclouds_pl': point_cloud,
#            'is_training_pl': is_training}
# #     feed_dict = {ops['pointclouds_pl']: [pc], ops['is_training_pl']: False}
#   feed_dict = {ops['pointclouds_pl']: [pc]}
#   batch_size_scores = sess.run(output, feed_dict=feed_dict)
#   print(batch_size_scores.shape)
#   print(batch_size_scores)
#   # 		print(VAl_CONSTANT+VAL_NUM_HEADING_BIN*NUM_HEADING_BIN+VAL_NUM_SIZE_CLUSTER*NUM_SIZE_CLUSTER)

In [0]:
##Generate random .csv file

# import random
# import csv

# with open('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/data_sample.csv', 'w', newline='') as writeFile:
# 	writer = csv.writer(writeFile)
# 	for x in range(1024):
# 		line = 	[0,0,0,0.0,0.0,0.0]
# 		line[0] = (random.randint(0,100))
# 		line[1] = (random.randint(0,100))
# 		line[2] = (random.randint(0,100))
# 		line[3] = (random.random())
# 		line[4] = (random.random())
# 		line[5] = (random.random())
# # 		print (line)
# 		writer.writerow(line)
# pc = np.loadtxt(open("/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/data_sample.csv", "rb"), delimiter=",")
# print (pc)